In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import requests
import numpy as np

In [3]:
API_KEY = os.getenv("EURI_API_KEY")

In [20]:
def generate_embeddings(text):
	url = "https://api.euron.one/api/v1/euri/embeddings"
	headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
	payload = {"input": text, "model": "text-embedding-3-small"}

	response = requests.post(url, headers=headers, json=payload)
	data = response.json()

	# convert to numpy array for vector operations
	embedding = np.array(data["data"][0]["embedding"])

	return embedding

In [10]:
text = "The weather is sunny today."

embedding = generate_embeddings(text)

In [11]:
embedding

array([ 0.01067466, -0.03126568,  0.00883722, ..., -0.03035571,
        0.01221461,  0.02594584], shape=(1536,))

In [12]:
print(f"Embedding shape: {embedding.shape}")

Embedding shape: (1536,)


In [13]:
# calculate vector norm
norm = np.linalg.norm(embedding)
print(f"vector norm: {norm}")

vector norm: 0.9999999991048422


In [5]:
def generate_completion():
	url = "https://api.euron.one/api/v1/euri/chat/completions"
	headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
	payload = {
		"messages": [
			{"role": "user", "content": "Write a poem about artificial intelligence"}
		],
		"model": "gpt-4.1-nano",
		"max_tokens": 500,
		"temperature": 0.7,
	}

	response = requests.post(url, headers=headers, json=payload)
	data = response.json()
	print(data)

In [28]:
generate_completion()

{'id': 'chatcmpl-77faab4b-a318-4be7-a5eb-8f34028b1715', 'object': 'chat.completion', 'created': 1757835996, 'model': 'deepseek-r1-distill-llama-70b', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '<think>\nOkay, so I need to write a poem about artificial intelligence. Hmm, where do I even start? Well, I know that AI is a big topic these days, so I should probably focus on some key aspects that people find interesting or thought-provoking. Maybe I can touch on how AI is created, its capabilities, and the impact it has on society.\n\nI remember that in the example poem, there were references to neural networks, circuits, and data streams. That makes sense because those are fundamental parts of how AI works. Maybe I can use similar imagery but put my own twist on it. I also noticed the example poem had a rhyme scheme, so I should consider that as well. Rhyming can make the poem flow better and be more engaging.\n\nLet me think about the structure. The example had fo

In [6]:
with open("../data/founder_story.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()

In [7]:
raw_text

"\nMaking an Impact\nHelping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.\n\nThe Entrepreneur and Teacher: Sudhanshu's Dual Legacy\nSudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Throughout his career, he has remaine

### Here chunk means it has approx. 100 words in each chunk

In [8]:
def chunk_text(text, max_words=100):
	chunks = []
	words = text.split()
	for i in range(0, len(words), max_words):
		chunks.append(" ".join(words[i : i + max_words]))
	return chunks

In [11]:
chunks = chunk_text(raw_text)
print(f"Total number of chunks: {len(chunks)}")
print(chunks[0])

Total number of chunks: 7
Making an Impact Helping Millions of Students Succeed Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances. In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after


In [21]:
embedding = generate_embeddings(chunks[0])
print(embedding)

[-0.02153425 -0.02938113  0.04458016 ... -0.02250996  0.01240246
  0.01720541]


In [18]:
import faiss

In [24]:
dimension = embedding.shape[0]
index = faiss.IndexFlatL2(dimension)

In [25]:
chunk_mapping = []
for chunk in chunks:
	embedding = generate_embeddings(chunk)
	index.add(np.array([embedding]).astype("float32"))
	chunk_mapping.append(chunk)

In [26]:
faiss.write_index(index, "../data/index.faiss")